# Aprendizagem de Máquina 2024.1
## Projeto final


## Questão 1: aprendizagem não-supervisionada
Fuzzy c-means com kernel Gaussiano, kernelização da métrica e cálculo dos parâmetros de janela

### Imports

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn import datasets
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score

#from ucimlrepo import fetch_ucirepo, list_available_datasets

from csv import reader
from math import sqrt
import random
from random import seed
from random import randrange
import requests
import io
    
# Downloading the csv file from your GitHub account

url = "https://raw.githubusercontent.com/Zuluke/Projetos-AM/main/Projeto_Chico/Dataset/mfeat-fac" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
fac = pd.read_csv(io.StringIO(download.decode('utf-8')),sep="\s+",header=None)

url = "https://raw.githubusercontent.com/Zuluke/Projetos-AM/main/Projeto_Chico/Dataset/mfeat-fou" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
fou = pd.read_csv(io.StringIO(download.decode('utf-8')),sep="\s+",header=None)

url = "https://raw.githubusercontent.com/Zuluke/Projetos-AM/main/Projeto_Chico/Dataset/mfeat-zer" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
zer = pd.read_csv(io.StringIO(download.decode('utf-8')),sep="\s+",header=None)

### Adjusting the datasets

In [2]:
# Label creation
fac["digit"] = [i for i in range(10) for __ in range(200)]

fou["digit"] = [i for i in range(10) for __ in range(200)]

zer["digit"] = [i for i in range(10) for __ in range(200)]

### Data preparation

In [3]:
target = "digit"


fac_x, fac_y = fac.drop(target, axis=1), fac[target]
#del fac

fou_x, fou_y = fou.drop(target, axis=1), fou[target]
#del fou

zer_x, zer_y = zer.drop(target, axis=1), zer[target]
#del zer

### zer

In [21]:
# tentando entender
h=np.random.rand(10,5)
print(h,'\n')
h=h[:, :, np.newaxis]
print(h,'\n')
f_idx = np.random.default_rng(10).integers(0, 3, 3)
f = h[f_idx]
print(f.T)

[[0.2661837  0.46652155 0.41050369 0.61194002 0.51986737]
 [0.9304142  0.14706032 0.33563919 0.46354194 0.90931049]
 [0.97747105 0.25812374 0.23597273 0.15563457 0.20399043]
 [0.30321203 0.78464698 0.90629004 0.01442324 0.13870777]
 [0.28201719 0.5588074  0.58327102 0.03988085 0.71011899]
 [0.95086092 0.83446201 0.02039596 0.20792198 0.18840412]
 [0.04355641 0.90170645 0.41969854 0.55723952 0.3557658 ]
 [0.46841096 0.04495932 0.74613503 0.72393293 0.76961119]
 [0.54612773 0.13264542 0.29012553 0.84433516 0.52368476]
 [0.59532568 0.44787316 0.06783319 0.02382516 0.27505312]] 

[[[0.2661837 ]
  [0.46652155]
  [0.41050369]
  [0.61194002]
  [0.51986737]]

 [[0.9304142 ]
  [0.14706032]
  [0.33563919]
  [0.46354194]
  [0.90931049]]

 [[0.97747105]
  [0.25812374]
  [0.23597273]
  [0.15563457]
  [0.20399043]]

 [[0.30321203]
  [0.78464698]
  [0.90629004]
  [0.01442324]
  [0.13870777]]

 [[0.28201719]
  [0.5588074 ]
  [0.58327102]
  [0.03988085]
  [0.71011899]]

 [[0.95086092]
  [0.83446201]
  

#### KFCM_K_W_1

In [4]:
class KFCM_K_W_1:
    def __init__(self, c, m=1.6, epochs=100, tol=1e-6, seed=0):
        # Hyperparameters
        self.m = m
        self.epochs = epochs
        self.tol = tol

        # c: number of clusters
        self.c = c

        self._zero = 1e-20
        self._rng = np.random.default_rng(seed)
        self._epoch = 0

        self._x = None
        self._y = None
        self._n = None
        self._p = None
        self._s = None
        self._u = None
        self._u_m = None
        self._u_m_kernel = None
        self._kernel = None
        self._denominator_j = None
        self._g = None
        self._j_new = None
        self._j_old = None
        return

    def fit(self, X, y=None):
        start_time = time()
        # Normalization
        scaler = MinMaxScaler()
        self._x = scaler.fit_transform(X)[:, :, np.newaxis]
        self._y = np.asarray(y)[:, np.newaxis]

        # n: number of instances
        # p: number of features
        self._n, self._p, _ = self._x.shape

        # Arrays initializations
        # s: array of width parameters 1/s^2, from Step 7
        self._s = np.ones((1,self._p))
        self._u = np.zeros((self._n, self.c, 1))

        # Prototype selection, Step 8
        g_idx = self._rng.integers(0, self._n, self.c)
        self._g = self._x[g_idx]

        self._update_kernel()

        # Compute the membership degree, Step 9,21
        self._update_u()

		# Compute the objective function, Step 11,23
        self._update_j()

        for epoch in range(1, self.epochs + 1):
            self._epoch = epoch
            # Step 13
            self._j_old = self._j_new

            # Step 15
            self._update_s()

            # Step 18
            self._update_g()

            # Step 21
            self._update_u()

            # Step 23
            self._update_j()
            if abs(self._j_new - self._j_old) < self.tol:
                break
        end_time = time()
        print(f"Execution time: {round((end_time - start_time) / 60, 2)} minutes")

        return

    def predict(self, X):
        return

    def evaluate(self, metric):
        metrics = {
            "accuracy": self._evaluate_accuracy,
            "MPC": self._evaluate_modified_partition_coefficient,
            "rand": self._evaluate_adjusted_rand_score,
            "error": self._evaluate_error,
        }
        metric_function = metrics.get(metric, "error")
        return metric_function()

    def _update_kernel(self):
        """Equation 10."""
        # Step 1: Calculate squared Euclidean distances
        squared_distances = (self._x - self._g.T) ** 2

        # Step 2: Apply weights
        weighted_distances = squared_distances * self._s.T

        # Step 3: Sum across dimensions
        summed_distances = weighted_distances.sum(axis=1)

        # Step 4: Apply the exponential function
        exponential = np.exp(summed_distances * (-1 / 2))
        self._kernel = exponential[:, :, np.newaxis]
        return

    def _update_u(self):
        """Equation 16a."""

        self._denominator_j = 2 - 2 * self._kernel
        self._denominator_j = np.maximum(self._denominator_j, self._zero)

        numerator = np.swapaxes(self._denominator_j, 1, 2)

        division = (numerator / self._denominator_j) ** (1 / (self.m - 1))

        self._u = division.sum(axis=1) ** -1
        self._u = self._u[:, :, np.newaxis]

        self._u_m = self._u**self.m
        self._u_m_kernel = self._u_m * self._kernel
        return

    def _update_j(self):
        """Equation 11."""
        self._j_new = (self._u_m * self._denominator_j).sum()
        print(f"Epoch: {self._epoch:03d} | Objective function J: {self._j_new:.8f}")
        return

    def _update_s(self):
        """Equation 14a."""
        squared_distances = (self._x - self._g.T) ** 2

        squared_distances = np.swapaxes(squared_distances, 1, 2)

        denominator = (self._u_m_kernel * squared_distances).sum(axis=0)[:, :, np.newaxis]
        
        denominator = (denominator).sum(axis=2)[:, :, np.newaxis] # Linha nova

        denominator = np.log(denominator)

        numerator = denominator.sum(axis=1)[:, :, np.newaxis] * self._p**-1

        subtracted = numerator - denominator

        self._s = np.exp(subtracted)

        self._update_kernel()
        return

    def _update_g(self):
        """Equation 15a."""
        x = np.swapaxes(self._x, 1, 2)

        numerator = (self._u_m_kernel * x).sum(axis=0)

        denominator = self._u_m_kernel.sum(axis=0)

        self._g = (numerator / denominator)[:, :, np.newaxis]

        self._update_kernel()
        return

    def _evaluate_accuracy(self):
        pred = np.argmax(self._u, axis=1)
        y_with_pred = pd.DataFrame(np.concatenate((self._y, pred), axis=1))
        y_with_pred["value"] = 1
        pivot_table = pd.pivot_table(
            y_with_pred, columns=[0], index=[1], values="value", aggfunc="sum"
        )
        pivot_table = pivot_table.fillna(0).values

        n_i = pivot_table.sum(axis=1)[:, np.newaxis]
        p_ij = pivot_table / n_i
        p_i = p_ij.max(axis=1)[:, np.newaxis]
        acc = (n_i * p_i).sum(axis=0) / self._n
        return acc[0]

    def _evaluate_modified_partition_coefficient(self):
        pc = np.sum(self._u**2) / self._n
        mpc = 1 - (self.c / (self.c - 1)) * (1 - pc)
        return mpc

    def _evaluate_adjusted_rand_score(self):
        y_pred = np.argmax(self._u, axis=1)
        return adjusted_rand_score(self._y[:, 0], y_pred[:, 0])

    def _evaluate_error(self):
        raise ValueError("Metric not implemented")

#### m = 1.3

In [15]:
# m = 1.3
u3 = []
g3 = []
s3 = []
j3 = []
acc_vec3 = []
mpc_vec3 = []
rand_vec3 = []
print("m = 1.3\n")
for run in range(50):
    model = KFCM_K_W_1(c=10, m=1.3, epochs=100, tol=1e-6, seed=run)
    model.fit(zer_x, zer_y)
    print(f"\nRun: {run+1}")
    acc = model.evaluate(metric="accuracy")
    print(f"Accuracy: {acc:.04f}")
    mpc = model.evaluate(metric="MPC")
    print(f"MPC: {mpc:.04f}")
    adjusted_rand_score_value = model.evaluate(metric="rand")
    print(f"Adjusted Rand Score: {adjusted_rand_score_value:.04f}\n")
    u3.append(model._u.reshape(2000, 10))
    g3.append(model._g.reshape(10, 47))
    s3.append(model._s.reshape(10, 47))
    j3.append(model._j_new)
    acc_vec3.append(acc)
    mpc_vec3.append(mpc)
    rand_vec3.append(adjusted_rand_score_value)

m = 1.3

Epoch: 000 | Objective function J: 1276.59927234
Epoch: 001 | Objective function J: 940.27813110
Epoch: 002 | Objective function J: 894.38589663
Epoch: 003 | Objective function J: 881.68582059
Epoch: 004 | Objective function J: 874.17622718
Epoch: 005 | Objective function J: 869.43543156
Epoch: 006 | Objective function J: 866.16350682
Epoch: 007 | Objective function J: 863.25483853
Epoch: 008 | Objective function J: 859.89837328
Epoch: 009 | Objective function J: 856.08432734
Epoch: 010 | Objective function J: 852.95013787
Epoch: 011 | Objective function J: 851.18370047
Epoch: 012 | Objective function J: 850.35308817
Epoch: 013 | Objective function J: 849.92048864
Epoch: 014 | Objective function J: 849.63439543
Epoch: 015 | Objective function J: 849.40716943
Epoch: 016 | Objective function J: 849.20771575
Epoch: 017 | Objective function J: 849.02313340
Epoch: 018 | Objective function J: 848.84721807
Epoch: 019 | Objective function J: 848.67678278
Epoch: 020 | Objective functio

In [16]:
# Best result for m = 1.3
best_j3 = np.argmin(j3)
print(f"Run: {best_j3+1}\nFinal J: {j3[best_j3]:.8f}\nAccuracy: {acc_vec3[best_j3]:.4f}")
print(f"MPC: {mpc_vec3[best_j3]:.4f}\nAdjusted Rand Score: {rand_vec3[best_j3]:.4f}")

Run: 42
Final J: 846.60113636
Accuracy: 0.4920
MPC: 0.3528
Adjusted Rand Score: 0.3052


In [17]:
# Metrics for m = 1.3
print(f"Mean Accuracy: {np.mean(acc_vec3):.4f}" + u" \u00b1 " + f"{np.std(acc_vec3):.4f}")
print(f"Mean Adjusted Rand Score: {np.mean(rand_vec3):.4f}" + u" \u00b1 " + f"{np.std(rand_vec3):.4f}")
print(f"Mean MPC: {np.mean(mpc_vec3):.4f}" + u" \u00b1 " + f"{np.std(mpc_vec3):.4f}")

Mean Accuracy: 0.4891 ± 0.0092
Mean Adjusted Rand Score: 0.3066 ± 0.0075
Mean MPC: 0.3487 ± 0.0072


In [20]:
# Crisp partition of the best result for m = 1.3
#best_u3 = pd.DataFrame(u3[best_j3])
best_u3 = np.array(u3[best_j3])
u3_crisp = np.zeros_like(best_u3)
for k in range(best_u3.shape[0]):
    max_index = np.argmax(best_u3[k])
    u3_crisp[k][max_index] = 1

In [21]:
# Confusion matrix for m = 1.3
num_groups = u3_crisp.shape[1]
clusters3 = np.zeros((num_groups, num_groups))
rows = u3_crisp.shape[0] // num_groups
for h in range(num_groups):
    clusters3[h] = np.sum(u3_crisp[h*rows:(h+1)*rows], axis=0)
clusters3 = pd.DataFrame(clusters3.astype(int))
clusters3

,0,1,2,3,4,5,6,7,8,9
0,1,0,95,0,2,0,0,98,1,3
1,0,80,2,0,26,84,0,5,1,2
2,6,0,7,1,7,3,67,2,3,104
3,8,0,2,7,28,7,106,0,10,32
4,5,49,0,7,79,56,0,1,3,0
5,7,2,4,1,15,8,35,28,37,63
6,107,0,5,4,11,10,5,3,50,5
7,1,0,0,161,21,0,1,0,16,0
8,9,0,105,0,2,0,4,64,8,8
9,117,1,3,1,14,8,4,1,46,5


#### m = 1.4

In [13]:
# m = 1.4
u4 = []
g4 = []
s4 = []
j4 = []
acc_vec4 = []
mpc_vec4 = []
rand_vec4 = []
print("m = 1.4\n")
for run in range(50):
    model = KFCM_K_W_2(c=10, m=1.4, epochs=100, tol=1e-6, seed=run)
    model.fit(x, y)
    print(f"\nRun: {run+1}")
    acc = model.evaluate(metric="accuracy")
    print(f"Accuracy: {acc:.04f}")
    mpc = model.evaluate(metric="MPC")
    print(f"MPC: {mpc:.04f}")
    adjusted_rand_score_value = model.evaluate(metric="rand")
    print(f"Adjusted Rand Score: {adjusted_rand_score_value:.04f}\n")
    u4.append(model._u.reshape(2000, 10))
    g4.append(model._g.reshape(10, 47))
    s4.append(model._s.reshape(10, 47))
    j4.append(model._j_new)
    acc_vec4.append(acc)
    mpc_vec4.append(mpc)
    rand_vec4.append(adjusted_rand_score_value)

m = 1.4

Epoch: 000 | Objective function J: 1037.72884733
Epoch: 001 | Objective function J: 770.79261616
Epoch: 002 | Objective function J: 736.33985479
Epoch: 003 | Objective function J: 727.96817498


/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 004 | Objective function J: 723.06109929
Epoch: 005 | Objective function J: 720.12397109
Epoch: 006 | Objective function J: 718.52355095
Epoch: 007 | Objective function J: 717.67273667
Epoch: 008 | Objective function J: 717.19573332
Epoch: 009 | Objective function J: 716.90360513
Epoch: 010 | Objective function J: 716.70706862
Epoch: 011 | Objective function J: 716.56191772
Epoch: 012 | Objective function J: 716.44377007
Epoch: 013 | Objective function J: 716.33688691
Epoch: 014 | Objective function J: 716.22878432
Epoch: 015 | Objective function J: 716.10705840
Epoch: 016 | Objective function J: 715.95692527
Epoch: 017 | Objective function J: 715.75900263
Epoch: 018 | Objective function J: 715.48792943
Epoch: 019 | Objective function J: 715.11445728
Epoch: 020 | Objective function J: 714.61662488
Epoch: 021 | Objective function J: 714.00431856
Epoch: 022 | Objective function J: 713.34320743
Epoch: 023 | Objective function J: 712.73857628
Epoch: 024 | Objective function J: 712.2

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 004 | Objective function J: 721.07635577
Epoch: 005 | Objective function J: 719.81031217
Epoch: 006 | Objective function J: 719.07003471
Epoch: 007 | Objective function J: 718.55740463
Epoch: 008 | Objective function J: 718.16158674
Epoch: 009 | Objective function J: 717.83423385
Epoch: 010 | Objective function J: 717.54646233
Epoch: 011 | Objective function J: 717.27337241
Epoch: 012 | Objective function J: 716.98819715
Epoch: 013 | Objective function J: 716.66043503
Epoch: 014 | Objective function J: 716.25771263
Epoch: 015 | Objective function J: 715.75474352
Epoch: 016 | Objective function J: 715.15276476
Epoch: 017 | Objective function J: 714.50075207
Epoch: 018 | Objective function J: 713.88868021
Epoch: 019 | Objective function J: 713.39785196
Epoch: 020 | Objective function J: 713.05444564
Epoch: 021 | Objective function J: 712.83378063
Epoch: 022 | Objective function J: 712.69411596
Epoch: 023 | Objective function J: 712.60045668
Epoch: 024 | Objective function J: 712.5

...

Epoch: 003 | Objective function J: 721.93421319
Epoch: 004 | Objective function J: 719.11739272
Epoch: 005 | Objective function J: 717.29527243
Epoch: 006 | Objective function J: 715.84871855
Epoch: 007 | Objective function J: 714.60906108
Epoch: 008 | Objective function J: 713.57359880
Epoch: 009 | Objective function J: 712.77344323
Epoch: 010 | Objective function J: 712.19931535
Epoch: 011 | Objective function J: 711.80425802
Epoch: 012 | Objective function J: 711.53539273
Epoch: 013 | Objective function J: 711.35126121
Epoch: 014 | Objective function J: 711.22356593
Epoch: 015 | Objective function J: 711.13380209
Epoch: 016 | Objective function J: 711.06989988
Epoch: 017 | Objective function J: 711.02389080
Epoch: 018 | Objective function J: 710.99042714
Epoch: 019 | Objective function J: 710.96586044
Epoch: 020 | Objective function J: 710.94766459
Epoch: 021 | Objective function J: 710.93406891
Epoch: 022 | Objective function J: 710.92381977
Epoch: 023 | Objective function J: 710.9

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 002 | Objective function J: 728.17096204
Epoch: 003 | Objective function J: 722.49260170
Epoch: 004 | Objective function J: 720.28327153
Epoch: 005 | Objective function J: 719.02348707
Epoch: 006 | Objective function J: 718.14779035
Epoch: 007 | Objective function J: 717.50912524
Epoch: 008 | Objective function J: 717.05834650
Epoch: 009 | Objective function J: 716.75601689
Epoch: 010 | Objective function J: 716.55995456
Epoch: 011 | Objective function J: 716.43323409
Epoch: 012 | Objective function J: 716.34925079
Epoch: 013 | Objective function J: 716.29103369
Epoch: 014 | Objective function J: 716.24836888
Epoch: 015 | Objective function J: 716.21516115
Epoch: 016 | Objective function J: 716.18765456
Epoch: 017 | Objective function J: 716.16336623
Epoch: 018 | Objective function J: 716.14046213
Epoch: 019 | Objective function J: 716.11736894
Epoch: 020 | Objective function J: 716.09249099
Epoch: 021 | Objective function J: 716.06394723
Epoch: 022 | Objective function J: 716.0

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 001 | Objective function J: 758.13317959
Epoch: 002 | Objective function J: 729.75585350
Epoch: 003 | Objective function J: 725.25177966
Epoch: 004 | Objective function J: 723.71584154
Epoch: 005 | Objective function J: 722.97702536
Epoch: 006 | Objective function J: 722.53964645
Epoch: 007 | Objective function J: 722.23024525
Epoch: 008 | Objective function J: 721.96382137
Epoch: 009 | Objective function J: 721.68375607
Epoch: 010 | Objective function J: 721.34352831
Epoch: 011 | Objective function J: 720.90671559
Epoch: 012 | Objective function J: 720.36135025
Epoch: 013 | Objective function J: 719.73615533
Epoch: 014 | Objective function J: 719.09216563
Epoch: 015 | Objective function J: 718.48677954
Epoch: 016 | Objective function J: 717.94804849
Epoch: 017 | Objective function J: 717.47879975
Epoch: 018 | Objective function J: 717.07136905
Epoch: 019 | Objective function J: 716.71608859
Epoch: 020 | Objective function J: 716.40286989
Epoch: 021 | Objective function J: 716.1

In [14]:
# Best result for m = 1.4
best_j4 = np.argmin(j4)
print(f"Run: {best_j4+1}\nFinal J: {j4[best_j4]:.8f}\nAccuracy: {acc_vec4[best_j4]:.4f}")
print(f"MPC: {mpc_vec4[best_j4]:.4f}\nAdjusted Rand Score: {rand_vec4[best_j4]:.4f}")

Run: 19
Final J: 710.87584761
Accuracy: 0.4560
MPC: 0.1704
Adjusted Rand Score: 0.2970


In [15]:
# Metrics for m = 1.4
print(f"Mean Accuracy: {np.mean(acc_vec4):.4f}" + u" \u00b1 " + f"{np.std(acc_vec4):.4f}")
print(f"Mean Adjusted Rand Score: {np.mean(rand_vec4):.4f}" + u" \u00b1 " + f"{np.std(rand_vec4):.4f}")
print(f"Mean MPC: {np.mean(mpc_vec4):.4f}" + u" \u00b1 " + f"{np.std(mpc_vec4):.4f}")

Mean Accuracy: 0.4506 ± 0.0046
Mean Adjusted Rand Score: 0.2972 ± 0.0029
Mean MPC: 0.1677 ± 0.0026


In [16]:
# Crisp partition of the best result for m = 1.4
#best_u4 = pd.DataFrame(u4[best_j4])
best_u4 = np.array(u4[best_j4])
u4_crisp = np.zeros_like(best_u4)
for k in range(best_u4.shape[0]):
    max_index = np.argmax(best_u4[k])
    u4_crisp[k][max_index] = 1

In [17]:
# Confusion matrix for m = 1.4
num_groups = u4_crisp.shape[1]
clusters4 = np.zeros((num_groups, num_groups))
rows = u4_crisp.shape[0] // num_groups
for h in range(num_groups):
    clusters4[h] = np.sum(u4_crisp[h*rows:(h+1)*rows], axis=0)
clusters4 = pd.DataFrame(clusters4.astype(int))
clusters4

,0,1,2,3,4,5,6,7,8,9
0,98,1,0,0,1,3,0,96,1,0
1,2,1,5,1,11,20,144,5,8,3
2,6,9,4,1,7,156,0,10,7,0
3,2,36,24,9,23,84,0,0,13,9
4,0,10,21,16,32,15,73,3,10,20
5,4,14,12,1,28,79,6,36,9,11
6,4,4,7,5,37,7,3,3,114,16
7,0,3,7,168,9,8,0,0,1,4
8,105,1,7,0,1,9,0,64,11,2
9,2,6,5,2,40,8,1,2,123,11


#### m = 1.5

In [18]:
# m = 1.5
u5 = []
g5 = []
s5 = []
j5 = []
acc_vec5 = []
mpc_vec5 = []
rand_vec5 = []
print("m = 1.5\n")
for run in range(50):
    model = KFCM_K_W_2(c=10, m=1.5, epochs=100, tol=1e-6, seed=run)
    model.fit(x, y)
    print(f"\nRun: {run+1}")
    acc = model.evaluate(metric="accuracy")
    print(f"Accuracy: {acc:.04f}")
    mpc = model.evaluate(metric="MPC")
    print(f"MPC: {mpc:.04f}")
    adjusted_rand_score_value = model.evaluate(metric="rand")
    print(f"Adjusted Rand Score: {adjusted_rand_score_value:.04f}\n")
    u5.append(model._u.reshape(2000, 10))
    g5.append(model._g.reshape(10, 47))
    s5.append(model._s.reshape(10, 47))
    j5.append(model._j_new)
    acc_vec5.append(acc)
    mpc_vec5.append(mpc)
    rand_vec5.append(adjusted_rand_score_value)

m = 1.5

Epoch: 000 | Objective function J: 836.53882344
Epoch: 001 | Objective function J: 623.47888858
Epoch: 002 | Objective function J: 596.21464752


/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 003 | Objective function J: 590.71750974
Epoch: 004 | Objective function J: 587.82027610
Epoch: 005 | Objective function J: 586.04323145
Epoch: 006 | Objective function J: 585.00151639
Epoch: 007 | Objective function J: 584.40656490
Epoch: 008 | Objective function J: 584.05247942
Epoch: 009 | Objective function J: 583.82472874
Epoch: 010 | Objective function J: 583.66680495
Epoch: 011 | Objective function J: 583.55060196
Epoch: 012 | Objective function J: 583.46100097
Epoch: 013 | Objective function J: 583.38908572
Epoch: 014 | Objective function J: 583.32920113
Epoch: 015 | Objective function J: 583.27760212
Epoch: 016 | Objective function J: 583.23177434
Epoch: 017 | Objective function J: 583.19005431
Epoch: 018 | Objective function J: 583.15138972
Epoch: 019 | Objective function J: 583.11516802
Epoch: 020 | Objective function J: 583.08108117
Epoch: 021 | Objective function J: 583.04901403
Epoch: 022 | Objective function J: 583.01895428
Epoch: 023 | Objective function J: 582.9

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 002 | Objective function J: 595.88027089
Epoch: 003 | Objective function J: 589.49791812
Epoch: 004 | Objective function J: 587.26984580
Epoch: 005 | Objective function J: 586.27938977
Epoch: 006 | Objective function J: 585.73101849
Epoch: 007 | Objective function J: 585.36462189
Epoch: 008 | Objective function J: 585.08557149
Epoch: 009 | Objective function J: 584.85501091
Epoch: 010 | Objective function J: 584.65506257
Epoch: 011 | Objective function J: 584.47672755
Epoch: 012 | Objective function J: 584.31505521
Epoch: 013 | Objective function J: 584.16690728
Epoch: 014 | Objective function J: 584.02987271
Epoch: 015 | Objective function J: 583.90184520
Epoch: 016 | Objective function J: 583.78101174
Epoch: 017 | Objective function J: 583.66604351
Epoch: 018 | Objective function J: 583.55630418
Epoch: 019 | Objective function J: 583.45193672
Epoch: 020 | Objective function J: 583.35375571
Epoch: 021 | Objective function J: 583.26295548
Epoch: 022 | Objective function J: 583.1

...

Epoch: 003 | Objective function J: 588.62951308
Epoch: 004 | Objective function J: 586.76580944
Epoch: 005 | Objective function J: 585.70479798
Epoch: 006 | Objective function J: 584.96260175
Epoch: 007 | Objective function J: 584.40418600
Epoch: 008 | Objective function J: 583.97873800
Epoch: 009 | Objective function J: 583.65421203
Epoch: 010 | Objective function J: 583.40329549
Epoch: 011 | Objective function J: 583.20415503
Epoch: 012 | Objective function J: 583.04177933
Epoch: 013 | Objective function J: 582.90708370
Epoch: 014 | Objective function J: 582.79492770
Epoch: 015 | Objective function J: 582.70223687
Epoch: 016 | Objective function J: 582.62671549
Epoch: 017 | Objective function J: 582.56618767
Epoch: 018 | Objective function J: 582.51840672
Epoch: 019 | Objective function J: 582.48112075
Epoch: 020 | Objective function J: 582.45221723
Epoch: 021 | Objective function J: 582.42984190
Epoch: 022 | Objective function J: 582.41245584
Epoch: 023 | Objective function J: 582.3

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 002 | Objective function J: 590.80667618
Epoch: 003 | Objective function J: 586.35668021
Epoch: 004 | Objective function J: 584.99029112
Epoch: 005 | Objective function J: 584.42138983
Epoch: 006 | Objective function J: 584.13031134
Epoch: 007 | Objective function J: 583.95080830
Epoch: 008 | Objective function J: 583.81939302
Epoch: 009 | Objective function J: 583.70954752
Epoch: 010 | Objective function J: 583.60994173
Epoch: 011 | Objective function J: 583.51618750
Epoch: 012 | Objective function J: 583.42737048
Epoch: 013 | Objective function J: 583.34434595
Epoch: 014 | Objective function J: 583.26867483
Epoch: 015 | Objective function J: 583.20183933
Epoch: 016 | Objective function J: 583.14471139
Epoch: 017 | Objective function J: 583.09732938
Epoch: 018 | Objective function J: 583.05897081
Epoch: 019 | Objective function J: 583.02841190
Epoch: 020 | Objective function J: 583.00423042
Epoch: 021 | Objective function J: 582.98504692
Epoch: 022 | Objective function J: 582.9

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 004 | Objective function J: 585.73641798
Epoch: 005 | Objective function J: 585.35185799
Epoch: 006 | Objective function J: 585.16223685
Epoch: 007 | Objective function J: 585.05374655
Epoch: 008 | Objective function J: 584.98295912
Epoch: 009 | Objective function J: 584.93109409
Epoch: 010 | Objective function J: 584.88921880
Epoch: 011 | Objective function J: 584.85274452
Epoch: 012 | Objective function J: 584.81916355
Epoch: 013 | Objective function J: 584.78704634
Epoch: 014 | Objective function J: 584.75556988
Epoch: 015 | Objective function J: 584.72428283
Epoch: 016 | Objective function J: 584.69297842
Epoch: 017 | Objective function J: 584.66161510
Epoch: 018 | Objective function J: 584.63025636
Epoch: 019 | Objective function J: 584.59901698
Epoch: 020 | Objective function J: 584.56801160
Epoch: 021 | Objective function J: 584.53730680
Epoch: 022 | Objective function J: 584.50688004
Epoch: 023 | Objective function J: 584.47658967
Epoch: 024 | Objective function J: 584.4

In [19]:
# Best result for m = 1.5
best_j5 = np.argmin(j5)
print(f"Run: {best_j5+1}\nFinal J: {j5[best_j5]:.8f}\nAccuracy: {acc_vec5[best_j5]:.4f}")
print(f"MPC: {mpc_vec5[best_j5]:.4f}\nAdjusted Rand Score: {rand_vec5[best_j5]:.4f}")

Run: 31
Final J: 581.96791567
Accuracy: 0.4340
MPC: 0.0943
Adjusted Rand Score: 0.2963


In [20]:
# Metrics for m = 1.5
print(f"Mean Accuracy: {np.mean(acc_vec5):.4f}" + u" \u00b1 " + f"{np.std(acc_vec5):.4f}")
print(f"Mean Adjusted Rand Score: {np.mean(rand_vec5):.4f}" + u" \u00b1 " + f"{np.std(rand_vec5):.4f}")
print(f"Mean MPC: {np.mean(mpc_vec5):.4f}" + u" \u00b1 " + f"{np.std(mpc_vec5):.4f}")

Mean Accuracy: 0.4196 ± 0.0158
Mean Adjusted Rand Score: 0.2769 ± 0.0229
Mean MPC: 0.0870 ± 0.0071


In [21]:
# Crisp partition of the best result for m = 1.5
#best_u5 = pd.DataFrame(u5[best_j5])
best_u5 = np.array(u5[best_j5])
u5_crisp = np.zeros_like(best_u5)
for k in range(best_u5.shape[0]):
    max_index = np.argmax(best_u5[k])
    u5_crisp[k][max_index] = 1

In [22]:
# Confusion matrix for m = 1.5
num_groups = u5_crisp.shape[1]
clusters5 = np.zeros((num_groups, num_groups))
rows = u5_crisp.shape[0] // num_groups
for h in range(num_groups):
    clusters5[h] = np.sum(u5_crisp[h*rows:(h+1)*rows], axis=0)
clusters5 = pd.DataFrame(clusters5.astype(int))
clusters5

,0,1,2,3,4,5,6,7,8,9
0,0,10,0,0,1,186,3,0,0,0
1,153,28,1,1,8,2,6,0,1,0
2,0,169,4,1,7,8,11,0,0,0
3,0,125,19,3,35,2,12,0,1,3
4,87,40,19,4,26,1,11,1,4,7
5,9,120,2,1,33,17,15,0,0,3
6,5,17,6,2,42,3,123,0,1,1
7,0,10,174,2,10,0,1,0,0,3
8,0,32,0,0,3,140,24,1,0,0
9,5,16,5,1,41,2,128,0,2,0


#### m = 1.6

In [23]:
# m = 1.6
u6 = []
g6 = []
s6 = []
j6 = []
acc_vec6 = []
mpc_vec6 = []
rand_vec6 = []
print("m = 1.6\n")
for run in range(50):
    model = KFCM_K_W_2(c=10, m=1.6, epochs=100, tol=1e-6, seed=run)
    model.fit(x, y)
    print(f"\nRun: {run+1}")
    acc = model.evaluate(metric="accuracy")
    print(f"Accuracy: {acc:.04f}")
    mpc = model.evaluate(metric="MPC")
    print(f"MPC: {mpc:.04f}")
    adjusted_rand_score_value = model.evaluate(metric="rand")
    print(f"Adjusted Rand Score: {adjusted_rand_score_value:.04f}\n")
    u6.append(model._u.reshape(2000, 10))
    g6.append(model._g.reshape(10, 47))
    s6.append(model._s.reshape(10, 47))
    j6.append(model._j_new)
    acc_vec6.append(acc)
    mpc_vec6.append(mpc)
    rand_vec6.append(adjusted_rand_score_value)

m = 1.6

Epoch: 000 | Objective function J: 671.21230539
Epoch: 001 | Objective function J: 500.94773878
Epoch: 002 | Objective function J: 478.76209279


/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 003 | Objective function J: 474.89256106
Epoch: 004 | Objective function J: 473.11848627
Epoch: 005 | Objective function J: 472.00517147
Epoch: 006 | Objective function J: 471.27791838
Epoch: 007 | Objective function J: 470.82043655
Epoch: 008 | Objective function J: 470.53874447
Epoch: 009 | Objective function J: 470.36129545
Epoch: 010 | Objective function J: 470.24343384
Epoch: 011 | Objective function J: 470.16039968
Epoch: 012 | Objective function J: 470.09885965
Epoch: 013 | Objective function J: 470.05139957
Epoch: 014 | Objective function J: 470.01363332
Epoch: 015 | Objective function J: 469.98278620
Epoch: 016 | Objective function J: 469.95699723
Epoch: 017 | Objective function J: 469.93495990
Epoch: 018 | Objective function J: 469.91572571
Epoch: 019 | Objective function J: 469.89858962
Epoch: 020 | Objective function J: 469.88301961
Epoch: 021 | Objective function J: 469.86861142
Epoch: 022 | Objective function J: 469.85505834
Epoch: 023 | Objective function J: 469.8

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 004 | Objective function J: 472.93772657
Epoch: 005 | Objective function J: 471.93389449
Epoch: 006 | Objective function J: 471.34910544
Epoch: 007 | Objective function J: 470.97612249
Epoch: 008 | Objective function J: 470.72388212
Epoch: 009 | Objective function J: 470.54524878
Epoch: 010 | Objective function J: 470.41366761
Epoch: 011 | Objective function J: 470.31340013
Epoch: 012 | Objective function J: 470.23472687
Epoch: 013 | Objective function J: 470.17141129
Epoch: 014 | Objective function J: 470.11930817
Epoch: 015 | Objective function J: 470.07557543
Epoch: 016 | Objective function J: 470.03821223
Epoch: 017 | Objective function J: 470.00577819
Epoch: 018 | Objective function J: 469.97721655
Epoch: 019 | Objective function J: 469.95173941
Epoch: 020 | Objective function J: 469.92875117
Epoch: 021 | Objective function J: 469.90779655
Epoch: 022 | Objective function J: 469.88852463
Epoch: 023 | Objective function J: 469.87066382
Epoch: 024 | Objective function J: 469.8

...

Epoch: 002 | Objective function J: 479.14070283
Epoch: 003 | Objective function J: 474.36541016
Epoch: 004 | Objective function J: 472.69248771
Epoch: 005 | Objective function J: 471.78795928
Epoch: 006 | Objective function J: 471.23011131
Epoch: 007 | Objective function J: 470.86714851
Epoch: 008 | Objective function J: 470.61914948
Epoch: 009 | Objective function J: 470.43939923
Epoch: 010 | Objective function J: 470.30113829
Epoch: 011 | Objective function J: 470.18970657
Epoch: 012 | Objective function J: 470.09733000
Epoch: 013 | Objective function J: 470.01985594
Epoch: 014 | Objective function J: 469.95485763
Epoch: 015 | Objective function J: 469.90062936
Epoch: 016 | Objective function J: 469.85572515
Epoch: 017 | Objective function J: 469.81879899
Epoch: 018 | Objective function J: 469.78858666
Epoch: 019 | Objective function J: 469.76393417
Epoch: 020 | Objective function J: 469.74382609
Epoch: 021 | Objective function J: 469.72739805
Epoch: 022 | Objective function J: 469.7

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 001 | Objective function J: 495.64736040
Epoch: 002 | Objective function J: 475.04880325
Epoch: 003 | Objective function J: 471.35100086
Epoch: 004 | Objective function J: 470.36829998
Epoch: 005 | Objective function J: 470.01635260
Epoch: 006 | Objective function J: 469.86201768
Epoch: 007 | Objective function J: 469.78037684
Epoch: 008 | Objective function J: 469.72894541
Epoch: 009 | Objective function J: 469.69180225
Epoch: 010 | Objective function J: 469.66247416
Epoch: 011 | Objective function J: 469.63807655
Epoch: 012 | Objective function J: 469.61717172
Epoch: 013 | Objective function J: 469.59894768
Epoch: 014 | Objective function J: 469.58288820
Epoch: 015 | Objective function J: 469.56863248
Epoch: 016 | Objective function J: 469.55591089
Epoch: 017 | Objective function J: 469.54451268
Epoch: 018 | Objective function J: 469.53426804
Epoch: 019 | Objective function J: 469.52503714
Epoch: 020 | Objective function J: 469.51670290
Epoch: 021 | Objective function J: 469.5

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 004 | Objective function J: 470.57018210
Epoch: 005 | Objective function J: 470.27878496
Epoch: 006 | Objective function J: 470.12647519
Epoch: 007 | Objective function J: 470.03464442
Epoch: 008 | Objective function J: 469.97333980
Epoch: 009 | Objective function J: 469.92939832
Epoch: 010 | Objective function J: 469.89630888
Epoch: 011 | Objective function J: 469.87049770
Epoch: 012 | Objective function J: 469.84982077
Epoch: 013 | Objective function J: 469.83289573
Epoch: 014 | Objective function J: 469.81878037
Epoch: 015 | Objective function J: 469.80680552
Epoch: 016 | Objective function J: 469.79648146
Epoch: 017 | Objective function J: 469.78744196
Epoch: 018 | Objective function J: 469.77940886
Epoch: 019 | Objective function J: 469.77216843
Epoch: 020 | Objective function J: 469.76555509
Epoch: 021 | Objective function J: 469.75943981
Epoch: 022 | Objective function J: 469.75372160
Epoch: 023 | Objective function J: 469.74832128
Epoch: 024 | Objective function J: 469.7

In [24]:
# Best result for m = 1.6
best_j6 = np.argmin(j6)
print(f"Run: {best_j6+1}\nFinal J: {j6[best_j6]:.8f}\nAccuracy: {acc_vec6[best_j6]:.4f}")
print(f"MPC: {mpc_vec6[best_j6]:.4f}\nAdjusted Rand Score: {rand_vec6[best_j6]:.4f}")

Run: 47
Final J: 469.38755502
Accuracy: 0.3970
MPC: 0.0411
Adjusted Rand Score: 0.2458


In [25]:
# Metrics for m = 1.6
print(f"Mean Accuracy: {np.mean(acc_vec6):.4f}" + u" \u00b1 " + f"{np.std(acc_vec6):.4f}")
print(f"Mean Adjusted Rand Score: {np.mean(rand_vec6):.4f}" + u" \u00b1 " + f"{np.std(rand_vec6):.4f}")
print(f"Mean MPC: {np.mean(mpc_vec6):.4f}" + u" \u00b1 " + f"{np.std(mpc_vec6):.4f}")

Mean Accuracy: 0.3582 ± 0.0273
Mean Adjusted Rand Score: 0.2165 ± 0.0249
Mean MPC: 0.0388 ± 0.0016


In [26]:
# Crisp partition of the best result for m = 1.6
#best_u6 = pd.DataFrame(u6[best_j6])
best_u6 = np.array(u6[best_j6])
u6_crisp = np.zeros_like(best_u6)
for k in range(best_u6.shape[0]):
    max_index = np.argmax(best_u6[k])
    u6_crisp[k][max_index] = 1

In [27]:
# Confusion matrix for m = 1.6
num_groups = u6_crisp.shape[1]
clusters6 = np.zeros((num_groups, num_groups))
rows = u6_crisp.shape[0] // num_groups
for h in range(num_groups):
    clusters6[h] = np.sum(u6_crisp[h*rows:(h+1)*rows], axis=0)
clusters6 = pd.DataFrame(clusters6.astype(int))
clusters6

,0,1,2,3,4,5,6,7,8,9
0,0,1,0,185,4,6,0,0,0,4
1,1,2,0,2,24,4,2,1,160,4
2,14,3,0,8,127,20,6,5,1,16
3,10,23,0,3,92,12,20,17,9,14
4,3,20,0,2,43,3,5,4,116,4
5,26,13,0,18,62,39,2,5,17,18
6,4,28,0,3,14,7,3,1,23,117
7,0,72,0,0,13,1,12,38,63,1
8,9,1,0,143,11,11,0,0,0,25
9,0,25,0,2,14,6,5,2,19,127


#### m = 1.7

In [28]:
# m = 1.7
u7 = []
g7 = []
s7 = []
j7 = []
acc_vec7 = []
mpc_vec7 = []
rand_vec7 = []
print("m = 1.7\n")
for run in range(50):
    model = KFCM_K_W_2(c=10, m=1.7, epochs=100, tol=1e-6, seed=run)
    model.fit(x, y)
    print(f"\nRun: {run+1}")
    acc = model.evaluate(metric="accuracy")
    print(f"Accuracy: {acc:.04f}")
    mpc = model.evaluate(metric="MPC")
    print(f"MPC: {mpc:.04f}")
    adjusted_rand_score_value = model.evaluate(metric="rand")
    print(f"Adjusted Rand Score: {adjusted_rand_score_value:.04f}\n")
    u7.append(model._u.reshape(2000, 10))
    g7.append(model._g.reshape(10, 47))
    s7.append(model._s.reshape(10, 47))
    j7.append(model._j_new)
    acc_vec7.append(acc)
    mpc_vec7.append(mpc)
    rand_vec7.append(adjusted_rand_score_value)

m = 1.7

Epoch: 000 | Objective function J: 537.03978285
Epoch: 001 | Objective function J: 401.09113503
Epoch: 002 | Objective function J: 382.76352566


/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 003 | Objective function J: 379.79326344
Epoch: 004 | Objective function J: 378.59900419
Epoch: 005 | Objective function J: 377.86454112
Epoch: 006 | Objective function J: 377.32996776
Epoch: 007 | Objective function J: 376.92997839
Epoch: 008 | Objective function J: 376.63766538
Epoch: 009 | Objective function J: 376.42846969
Epoch: 010 | Objective function J: 376.27877048
Epoch: 011 | Objective function J: 376.16972827
Epoch: 012 | Objective function J: 376.08823951
Epoch: 013 | Objective function J: 376.02577077
Epoch: 014 | Objective function J: 375.97683529
Epoch: 015 | Objective function J: 375.93783464
Epoch: 016 | Objective function J: 375.90632427
Epoch: 017 | Objective function J: 375.88057880
Epoch: 018 | Objective function J: 375.85933792
Epoch: 019 | Objective function J: 375.84165526
Epoch: 020 | Objective function J: 375.82680531
Epoch: 021 | Objective function J: 375.81422353
Epoch: 022 | Objective function J: 375.80346605
Epoch: 023 | Objective function J: 375.7

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 004 | Objective function J: 378.61429504
Epoch: 005 | Objective function J: 377.69906183
Epoch: 006 | Objective function J: 377.14750531
Epoch: 007 | Objective function J: 376.78899228
Epoch: 008 | Objective function J: 376.54461836
Epoch: 009 | Objective function J: 376.37190820
Epoch: 010 | Objective function J: 376.24624909
Epoch: 011 | Objective function J: 376.15265498
Epoch: 012 | Objective function J: 376.08160467
Epoch: 013 | Objective function J: 376.02681329
Epoch: 014 | Objective function J: 375.98399132
Epoch: 015 | Objective function J: 375.95012731
Epoch: 016 | Objective function J: 375.92305707
Epoch: 017 | Objective function J: 375.90119526
Epoch: 018 | Objective function J: 375.88336225
Epoch: 019 | Objective function J: 375.86866889
Epoch: 020 | Objective function J: 375.85643773
Epoch: 021 | Objective function J: 375.84614789
Epoch: 022 | Objective function J: 375.83739573
Epoch: 023 | Objective function J: 375.82986631
Epoch: 024 | Objective function J: 375.8

...

Epoch: 003 | Objective function J: 379.99598997
Epoch: 004 | Objective function J: 378.41399904
Epoch: 005 | Objective function J: 377.54634541
Epoch: 006 | Objective function J: 377.00820913
Epoch: 007 | Objective function J: 376.65403333
Epoch: 008 | Objective function J: 376.41167071
Epoch: 009 | Objective function J: 376.24156019
Epoch: 010 | Objective function J: 376.12012681
Epoch: 011 | Objective function J: 376.03222009
Epoch: 012 | Objective function J: 375.96764436
Epoch: 013 | Objective function J: 375.91941452
Epoch: 014 | Objective function J: 375.88272591
Epoch: 015 | Objective function J: 375.85426882
Epoch: 016 | Objective function J: 375.83175411
Epoch: 017 | Objective function J: 375.81358620
Epoch: 018 | Objective function J: 375.79864090
Epoch: 019 | Objective function J: 375.78611614
Epoch: 020 | Objective function J: 375.77543189
Epoch: 021 | Objective function J: 375.76616288
Epoch: 022 | Objective function J: 375.75799296
Epoch: 023 | Objective function J: 375.7

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 000 | Objective function J: 520.76215020
Epoch: 001 | Objective function J: 397.76830844
Epoch: 002 | Objective function J: 380.22536357
Epoch: 003 | Objective function J: 377.10190948
Epoch: 004 | Objective function J: 376.32969634
Epoch: 005 | Objective function J: 376.04352316
Epoch: 006 | Objective function J: 375.89990198
Epoch: 007 | Objective function J: 375.81022259
Epoch: 008 | Objective function J: 375.74604003
Epoch: 009 | Objective function J: 375.69646880
Epoch: 010 | Objective function J: 375.65657070
Epoch: 011 | Objective function J: 375.62370239
Epoch: 012 | Objective function J: 375.59623353
Epoch: 013 | Objective function J: 375.57304872
Epoch: 014 | Objective function J: 375.55333169
Epoch: 015 | Objective function J: 375.53645960
Epoch: 016 | Objective function J: 375.52194453
Epoch: 017 | Objective function J: 375.50939754
Epoch: 018 | Objective function J: 375.49850467
Epoch: 019 | Objective function J: 375.48900998
Epoch: 020 | Objective function J: 375.4

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 004 | Objective function J: 376.49767414
Epoch: 005 | Objective function J: 376.24384806
Epoch: 006 | Objective function J: 376.09717644
Epoch: 007 | Objective function J: 376.00138306
Epoch: 008 | Objective function J: 375.93404991
Epoch: 009 | Objective function J: 375.88440963
Epoch: 010 | Objective function J: 375.84654689
Epoch: 011 | Objective function J: 375.81688312
Epoch: 012 | Objective function J: 375.79310032
Epoch: 013 | Objective function J: 375.77362249
Epoch: 014 | Objective function J: 375.75733993
Epoch: 015 | Objective function J: 375.74345069
Epoch: 016 | Objective function J: 375.73136331
Epoch: 017 | Objective function J: 375.72063420
Epoch: 018 | Objective function J: 375.71092580
Epoch: 019 | Objective function J: 375.70197770
Epoch: 020 | Objective function J: 375.69358652
Epoch: 021 | Objective function J: 375.68559140
Epoch: 022 | Objective function J: 375.67786373
Epoch: 023 | Objective function J: 375.67029966
Epoch: 024 | Objective function J: 375.6

In [29]:
# Best result for m = 1.7
best_j7 = np.argmin(j7)
print(f"Run: {best_j7+1}\nFinal J: {j7[best_j7]:.8f}\nAccuracy: {acc_vec7[best_j7]:.4f}")
print(f"MPC: {mpc_vec7[best_j7]:.4f}\nAdjusted Rand Score: {rand_vec7[best_j7]:.4f}")

Run: 24
Final J: 375.39394783
Accuracy: 0.2730
MPC: 0.0208
Adjusted Rand Score: 0.1604


In [30]:
# Metrics for m = 1.7
print(f"Mean Accuracy: {np.mean(acc_vec7):.4f}" + u" \u00b1 " + f"{np.std(acc_vec7):.4f}")
print(f"Mean Adjusted Rand Score: {np.mean(rand_vec7):.4f}" + u" \u00b1 " + f"{np.std(rand_vec7):.4f}")
print(f"Mean MPC: {np.mean(mpc_vec7):.4f}" + u" \u00b1 " + f"{np.std(mpc_vec7):.4f}")

Mean Accuracy: 0.2879 ± 0.0176
Mean Adjusted Rand Score: 0.1701 ± 0.0112
Mean MPC: 0.0210 ± 0.0002


In [31]:
# Crisp partition of the best result for m = 1.7
#best_u7 = pd.DataFrame(u7[best_j7])
best_u7 = np.array(u7[best_j7])
u7_crisp = np.zeros_like(best_u7)
for k in range(best_u7.shape[0]):
    max_index = np.argmax(best_u7[k])
    u7_crisp[k][max_index] = 1

In [32]:
# Confusion matrix for m = 1.7
num_groups = u7_crisp.shape[1]
clusters7 = np.zeros((num_groups, num_groups))
rows = u7_crisp.shape[0] // num_groups
for h in range(num_groups):
    clusters7[h] = np.sum(u7_crisp[h*rows:(h+1)*rows], axis=0)
clusters7 = pd.DataFrame(clusters7.astype(int))
clusters7

,0,1,2,3,4,5,6,7,8,9
0,187,0,8,1,0,0,0,4,0,0
1,3,0,33,1,0,1,0,162,0,0
2,13,0,162,3,2,0,0,20,0,0
3,3,0,167,0,1,1,0,28,0,0
4,3,0,98,1,0,0,0,91,7,0
5,25,0,129,2,3,0,0,40,1,0
6,13,0,39,1,0,0,0,146,1,0
7,0,0,183,0,0,0,0,14,3,0
8,160,0,24,1,1,0,0,14,0,0
9,10,0,35,2,0,0,0,151,2,0


#### m = 1.8

In [33]:
# m = 1.8
u8 = []
g8 = []
s8 = []
j8 = []
acc_vec8 = []
mpc_vec8 = []
rand_vec8 = []
print("m = 1.8\n")
for run in range(50):
    model = KFCM_K_W_2(c=10, m=1.8, epochs=100, tol=1e-6, seed=run)
    model.fit(x, y)
    print(f"\nRun: {run+1}")
    acc = model.evaluate(metric="accuracy")
    print(f"Accuracy: {acc:.04f}")
    mpc = model.evaluate(metric="MPC")
    print(f"MPC: {mpc:.04f}")
    adjusted_rand_score_value = model.evaluate(metric="rand")
    print(f"Adjusted Rand Score: {adjusted_rand_score_value:.04f}\n")
    u8.append(model._u.reshape(2000, 10))
    g8.append(model._g.reshape(10, 47))
    s8.append(model._s.reshape(10, 47))
    j8.append(model._j_new)
    acc_vec8.append(acc)
    mpc_vec8.append(mpc)
    rand_vec8.append(adjusted_rand_score_value)

m = 1.8

Epoch: 000 | Objective function J: 428.90701107
Epoch: 001 | Objective function J: 320.55619477
Epoch: 002 | Objective function J: 305.28156328
Epoch: 003 | Objective function J: 302.85527761
Epoch: 004 | Objective function J: 301.97206169


/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 005 | Objective function J: 301.47263919
Epoch: 006 | Objective function J: 301.11364923
Epoch: 007 | Objective function J: 300.82836718
Epoch: 008 | Objective function J: 300.59823648
Epoch: 009 | Objective function J: 300.41658508
Epoch: 010 | Objective function J: 300.27742658
Epoch: 011 | Objective function J: 300.17336956
Epoch: 012 | Objective function J: 300.09660502
Epoch: 013 | Objective function J: 300.04014284
Epoch: 014 | Objective function J: 299.99839511
Epoch: 015 | Objective function J: 299.96719001
Epoch: 016 | Objective function J: 299.94352430
Epoch: 017 | Objective function J: 299.92526859
Epoch: 018 | Objective function J: 299.91091636
Epoch: 019 | Objective function J: 299.89939617
Epoch: 020 | Objective function J: 299.88993928
Epoch: 021 | Objective function J: 299.88198825
Epoch: 022 | Objective function J: 299.87513436
Epoch: 023 | Objective function J: 299.86907440
Epoch: 024 | Objective function J: 299.86358060
Epoch: 025 | Objective function J: 299.8

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 006 | Objective function J: 300.93472223
Epoch: 007 | Objective function J: 300.63495811
Epoch: 008 | Objective function J: 300.42982697
Epoch: 009 | Objective function J: 300.28573223
Epoch: 010 | Objective function J: 300.18258167
Epoch: 011 | Objective function J: 300.10757352
Epoch: 012 | Objective function J: 300.05224788
Epoch: 013 | Objective function J: 300.01088077
Epoch: 014 | Objective function J: 299.97953023
Epoch: 015 | Objective function J: 299.95544106
Epoch: 016 | Objective function J: 299.93666224
Epoch: 017 | Objective function J: 299.92179590
Epoch: 018 | Objective function J: 299.90982960
Epoch: 019 | Objective function J: 299.90002267
Epoch: 020 | Objective function J: 299.89182795
Epoch: 021 | Objective function J: 299.88483735
Epoch: 022 | Objective function J: 299.87874355
Epoch: 023 | Objective function J: 299.87331269
Epoch: 024 | Objective function J: 299.86836489
Epoch: 025 | Objective function J: 299.86376010
Epoch: 026 | Objective function J: 299.8

...

Epoch: 002 | Objective function J: 307.30826714
Epoch: 003 | Objective function J: 303.42770947
Epoch: 004 | Objective function J: 301.99313163
Epoch: 005 | Objective function J: 301.22340455
Epoch: 006 | Objective function J: 300.76552621
Epoch: 007 | Objective function J: 300.47908936
Epoch: 008 | Objective function J: 300.29283435
Epoch: 009 | Objective function J: 300.16729980
Epoch: 010 | Objective function J: 300.07964148
Epoch: 011 | Objective function J: 300.01624929
Epoch: 012 | Objective function J: 299.96881293
Epoch: 013 | Objective function J: 299.93212525
Epoch: 014 | Objective function J: 299.90283052
Epoch: 015 | Objective function J: 299.87870213
Epoch: 016 | Objective function J: 299.85821948
Epoch: 017 | Objective function J: 299.84031505
Epoch: 018 | Objective function J: 299.82421973
Epoch: 019 | Objective function J: 299.80936601
Epoch: 020 | Objective function J: 299.79532628
Epoch: 021 | Objective function J: 299.78177308
Epoch: 022 | Objective function J: 299.7

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 005 | Objective function J: 300.05265270
Epoch: 006 | Objective function J: 299.91712720
Epoch: 007 | Objective function J: 299.82566331
Epoch: 008 | Objective function J: 299.75777658
Epoch: 009 | Objective function J: 299.70498825
Epoch: 010 | Objective function J: 299.66288183
Epoch: 011 | Objective function J: 299.62874963
Epoch: 012 | Objective function J: 299.60075721
Epoch: 013 | Objective function J: 299.57758709
Epoch: 014 | Objective function J: 299.55825918
Epoch: 015 | Objective function J: 299.54202743
Epoch: 016 | Objective function J: 299.52831464
Epoch: 017 | Objective function J: 299.51666861
Epoch: 018 | Objective function J: 299.50673136
Epoch: 019 | Objective function J: 299.49821700
Epoch: 020 | Objective function J: 299.49089530
Epoch: 021 | Objective function J: 299.48457944
Epoch: 022 | Objective function J: 299.47911671
Epoch: 023 | Objective function J: 299.47438136
Epoch: 024 | Objective function J: 299.47026914
Epoch: 025 | Objective function J: 299.4

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 005 | Objective function J: 300.26299916
Epoch: 006 | Objective function J: 300.12522762
Epoch: 007 | Objective function J: 300.03043828
Epoch: 008 | Objective function J: 299.96079147
Epoch: 009 | Objective function J: 299.90718111
Epoch: 010 | Objective function J: 299.86429117
Epoch: 011 | Objective function J: 299.82875936
Epoch: 012 | Objective function J: 299.79835713
Epoch: 013 | Objective function J: 299.77156807
Epoch: 014 | Objective function J: 299.74734888
Epoch: 015 | Objective function J: 299.72498414
Epoch: 016 | Objective function J: 299.70399327
Epoch: 017 | Objective function J: 299.68406797
Epoch: 018 | Objective function J: 299.66502742
Epoch: 019 | Objective function J: 299.64678423
Epoch: 020 | Objective function J: 299.62931695
Epoch: 021 | Objective function J: 299.61264693
Epoch: 022 | Objective function J: 299.59681894
Epoch: 023 | Objective function J: 299.58188526
Epoch: 024 | Objective function J: 299.56789367
Epoch: 025 | Objective function J: 299.5

In [34]:
# Best result for m = 1.8
best_j8 = np.argmin(j8)
print(f"Run: {best_j8+1}\nFinal J: {j8[best_j8]:.8f}\nAccuracy: {acc_vec8[best_j8]:.4f}")
print(f"MPC: {mpc_vec8[best_j8]:.4f}\nAdjusted Rand Score: {rand_vec8[best_j8]:.4f}")

Run: 11
Final J: 299.44282829
Accuracy: 0.2795
MPC: 0.0138
Adjusted Rand Score: 0.1559


In [35]:
# Metrics for m = 1.8
print(f"Mean Accuracy: {np.mean(acc_vec8):.4f}" + u" \u00b1 " + f"{np.std(acc_vec8):.4f}")
print(f"Mean Adjusted Rand Score: {np.mean(rand_vec8):.4f}" + u" \u00b1 " + f"{np.std(rand_vec8):.4f}")
print(f"Mean MPC: {np.mean(mpc_vec8):.4f}" + u" \u00b1 " + f"{np.std(mpc_vec8):.4f}")

Mean Accuracy: 0.2841 ± 0.0132
Mean Adjusted Rand Score: 0.1613 ± 0.0092
Mean MPC: 0.0138 ± 0.0000


In [36]:
# Crisp partition of the best result for m = 1.8
#best_u8 = pd.DataFrame(u8[best_j8])
best_u8 = np.array(u8[best_j8])
u8_crisp = np.zeros_like(best_u8)
for k in range(best_u8.shape[0]):
    max_index = np.argmax(best_u8[k])
    u8_crisp[k][max_index] = 1

In [37]:
# Confusion matrix for m = 1.8
num_groups = u8_crisp.shape[1]
clusters8 = np.zeros((num_groups, num_groups))
rows = u8_crisp.shape[0] // num_groups
for h in range(num_groups):
    clusters8[h] = np.sum(u8_crisp[h*rows:(h+1)*rows], axis=0)
clusters8 = pd.DataFrame(clusters8.astype(int))
clusters8

,0,1,2,3,4,5,6,7,8,9
0,4,0,0,0,0,0,0,1,188,7
1,44,0,0,0,0,0,7,0,3,146
2,154,0,0,0,0,0,0,1,13,32
3,166,1,0,0,0,0,1,2,3,27
4,115,0,0,0,0,0,20,1,3,61
5,110,0,0,0,0,0,0,11,25,54
6,40,0,0,0,0,0,3,0,16,141
7,193,0,0,0,0,0,4,1,0,2
8,19,0,0,0,0,0,0,1,161,19
9,38,0,0,0,0,0,4,1,11,146


#### m = 1.9

In [38]:
# m = 1.9
u9 = []
g9 = []
s9 = []
j9 = []
acc_vec9 = []
mpc_vec9 = []
rand_vec9 = []
print("m = 1.9\n")
for run in range(50):
    model = KFCM_K_W_2(c=10, m=1.9, epochs=100, tol=1e-6, seed=run)
    model.fit(x, y)
    print(f"\nRun: {run+1}")
    acc = model.evaluate(metric="accuracy")
    print(f"Accuracy: {acc:.04f}")
    mpc = model.evaluate(metric="MPC")
    print(f"MPC: {mpc:.04f}")
    adjusted_rand_score_value = model.evaluate(metric="rand")
    print(f"Adjusted Rand Score: {adjusted_rand_score_value:.04f}\n")
    u9.append(model._u.reshape(2000, 10))
    g9.append(model._g.reshape(10, 47))
    s9.append(model._s.reshape(10, 47))
    j9.append(model._j_new)
    acc_vec9.append(acc)
    mpc_vec9.append(mpc)
    rand_vec9.append(adjusted_rand_score_value)

m = 1.9

Epoch: 000 | Objective function J: 342.12597390
Epoch: 001 | Objective function J: 255.97624365
Epoch: 002 | Objective function J: 243.16513017
Epoch: 003 | Objective function J: 241.11365719
Epoch: 004 | Objective function J: 240.41355736


/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 005 | Objective function J: 240.05249460
Epoch: 006 | Objective function J: 239.81682463
Epoch: 007 | Objective function J: 239.64133108
Epoch: 008 | Objective function J: 239.50185816
Epoch: 009 | Objective function J: 239.38802570
Epoch: 010 | Objective function J: 239.29479066
Epoch: 011 | Objective function J: 239.21920174
Epoch: 012 | Objective function J: 239.15895064
Epoch: 013 | Objective function J: 239.11178890
Epoch: 014 | Objective function J: 239.07542129
Epoch: 015 | Objective function J: 239.04761375
Epoch: 016 | Objective function J: 239.02633832
Epoch: 017 | Objective function J: 239.00986871
Epoch: 018 | Objective function J: 238.99680866
Epoch: 019 | Objective function J: 238.98607012
Epoch: 020 | Objective function J: 238.97682541
Epoch: 021 | Objective function J: 238.96845239
Epoch: 022 | Objective function J: 238.96048381
Epoch: 023 | Objective function J: 238.95256528
Epoch: 024 | Objective function J: 238.94442294
Epoch: 025 | Objective function J: 238.9

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 003 | Objective function J: 241.91545405
Epoch: 004 | Objective function J: 240.68933467
Epoch: 005 | Objective function J: 240.08685693
Epoch: 006 | Objective function J: 239.73224291
Epoch: 007 | Objective function J: 239.50373821
Epoch: 008 | Objective function J: 239.34908073
Epoch: 009 | Objective function J: 239.24132053
Epoch: 010 | Objective function J: 239.16474582
Epoch: 011 | Objective function J: 239.10942957
Epoch: 012 | Objective function J: 239.06877782
Epoch: 013 | Objective function J: 239.03827698
Epoch: 014 | Objective function J: 239.01478034
Epoch: 015 | Objective function J: 238.99606654
Epoch: 016 | Objective function J: 238.98055101
Epoch: 017 | Objective function J: 238.96709141
Epoch: 018 | Objective function J: 238.95485422
Epoch: 019 | Objective function J: 238.94322292
Epoch: 020 | Objective function J: 238.93173499
Epoch: 021 | Objective function J: 238.92003949
Epoch: 022 | Objective function J: 238.90786949
Epoch: 023 | Objective function J: 238.8

...

Epoch: 002 | Objective function J: 245.29865216
Epoch: 003 | Objective function J: 241.85292373
Epoch: 004 | Objective function J: 240.59853829
Epoch: 005 | Objective function J: 239.95245401
Epoch: 006 | Objective function J: 239.58757451
Epoch: 007 | Objective function J: 239.36962529
Epoch: 008 | Objective function J: 239.23208588
Epoch: 009 | Objective function J: 239.14030927
Epoch: 010 | Objective function J: 239.07564780
Epoch: 011 | Objective function J: 239.02765491
Epoch: 012 | Objective function J: 238.99019611
Epoch: 013 | Objective function J: 238.95949266
Epoch: 014 | Objective function J: 238.93311766
Epoch: 015 | Objective function J: 238.90946502
Epoch: 016 | Objective function J: 238.88745809
Epoch: 017 | Objective function J: 238.86638407
Epoch: 018 | Objective function J: 238.84579594
Epoch: 019 | Objective function J: 238.82545099
Epoch: 020 | Objective function J: 238.80526771
Epoch: 021 | Objective function J: 238.78529042
Epoch: 022 | Objective function J: 238.7

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 003 | Objective function J: 239.77468532
Epoch: 004 | Objective function J: 239.26092046
Epoch: 005 | Objective function J: 239.05704537
Epoch: 006 | Objective function J: 238.94021920
Epoch: 007 | Objective function J: 238.86036573
Epoch: 008 | Objective function J: 238.80150355
Epoch: 009 | Objective function J: 238.75659494
Epoch: 010 | Objective function J: 238.72169020
Epoch: 011 | Objective function J: 238.69423477
Epoch: 012 | Objective function J: 238.67245054
Epoch: 013 | Objective function J: 238.65505023
Epoch: 014 | Objective function J: 238.64107903
Epoch: 015 | Objective function J: 238.62981614
Epoch: 016 | Objective function J: 238.62070945
Epoch: 017 | Objective function J: 238.61333076
Epoch: 018 | Objective function J: 238.60734421
Epoch: 019 | Objective function J: 238.60248375
Epoch: 020 | Objective function J: 238.59853690
Epoch: 021 | Objective function J: 238.59533273
Epoch: 022 | Objective function J: 238.59273301
Epoch: 023 | Objective function J: 238.5

/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/e-bach/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Sp

Epoch: 000 | Objective function J: 328.16671753
Epoch: 001 | Objective function J: 252.09437645
Epoch: 002 | Objective function J: 241.23267506
Epoch: 003 | Objective function J: 239.83634136
Epoch: 004 | Objective function J: 239.44733198
Epoch: 005 | Objective function J: 239.25207793
Epoch: 006 | Objective function J: 239.12930773
Epoch: 007 | Objective function J: 239.04303816
Epoch: 008 | Objective function J: 238.97817196
Epoch: 009 | Objective function J: 238.92689522
Epoch: 010 | Objective function J: 238.88465999
Epoch: 011 | Objective function J: 238.84866762
Epoch: 012 | Objective function J: 238.81716745
Epoch: 013 | Objective function J: 238.78907528
Epoch: 014 | Objective function J: 238.76373851
Epoch: 015 | Objective function J: 238.74077878
Epoch: 016 | Objective function J: 238.71998179
Epoch: 017 | Objective function J: 238.70122038
Epoch: 018 | Objective function J: 238.68440337
Epoch: 019 | Objective function J: 238.66944454
Epoch: 020 | Objective function J: 238.6

In [39]:
# Best result for m = 1.9
best_j9 = np.argmin(j9)
print(f"Run: {best_j9+1}\nFinal J: {j9[best_j9]:.8f}\nAccuracy: {acc_vec9[best_j9]:.4f}")
print(f"MPC: {mpc_vec9[best_j9]:.4f}\nAdjusted Rand Score: {rand_vec9[best_j9]:.4f}")

Run: 24
Final J: 238.58185102
Accuracy: 0.3365
MPC: 0.0093
Adjusted Rand Score: 0.1798


In [40]:
# Metrics for m = 1.9
print(f"Mean Accuracy: {np.mean(acc_vec9):.4f}" + u" \u00b1 " + f"{np.std(acc_vec9):.4f}")
print(f"Mean Adjusted Rand Score: {np.mean(rand_vec9):.4f}" + u" \u00b1 " + f"{np.std(rand_vec9):.4f}")
print(f"Mean MPC: {np.mean(mpc_vec9):.4f}" + u" \u00b1 " + f"{np.std(mpc_vec9):.4f}")

Mean Accuracy: 0.3028 ± 0.0190
Mean Adjusted Rand Score: 0.1612 ± 0.0145
Mean MPC: 0.0093 ± 0.0000


In [41]:
# Crisp partition of the best result for m = 1.9
#best_u9 = pd.DataFrame(u9[best_j9])
best_u9 = np.array(u9[best_j9])
u9_crisp = np.zeros_like(best_u9)
for k in range(best_u9.shape[0]):
    max_index = np.argmax(best_u9[k])
    u9_crisp[k][max_index] = 1

In [58]:
# Confusion matrix for m = 1.9
num_groups = u9_crisp.shape[1]
clusters9 = np.zeros((num_groups, num_groups))
rows = u9_crisp.shape[0] // num_groups
for h in range(num_groups):
    clusters9[h] = np.sum(u9_crisp[h*rows:(h+1)*rows], axis=0)
clusters9 = pd.DataFrame(clusters9.astype(int))
clusters9

,0,1,2,3,4,5,6,7,8,9
0,188,0,4,8,0,0,0,0,0,0
1,2,0,33,25,0,0,0,99,41,0
2,14,0,147,32,6,0,0,0,1,0
3,3,0,160,23,6,0,0,4,4,0
4,2,0,99,7,1,0,0,36,55,0
5,26,0,101,46,15,0,0,7,5,0
6,20,0,38,117,1,0,0,17,7,0
7,0,0,183,0,1,0,0,0,16,0
8,163,0,18,17,2,0,0,0,0,0
9,13,0,35,133,0,0,0,10,9,0


#### Best

In [59]:
# Save confusion matrices
with pd.ExcelWriter("confusion-zer.xlsx") as writer:
    clusters3.to_excel(writer, sheet_name="m=1.3")
    clusters4.to_excel(writer, sheet_name="m=1.4")
    clusters5.to_excel(writer, sheet_name="m=1.5")
    clusters6.to_excel(writer, sheet_name="m=1.6")
    clusters7.to_excel(writer, sheet_name="m=1.7")
    clusters8.to_excel(writer, sheet_name="m=1.8")
    clusters9.to_excel(writer, sheet_name="m=1.9")

In [74]:
# Cluster representatives
best_g3 = np.array(g3[best_j3])
best_g3 = pd.DataFrame(best_g3)
best_g3 = best_g3.round(4)

best_g4 = np.array(g4[best_j4])
best_g4 = pd.DataFrame(best_g4)
best_g4 = best_g4.round(4)

best_g5 = np.array(g5[best_j5])
best_g5 = pd.DataFrame(best_g5)
best_g5 = best_g5.round(4)

best_g6 = np.array(g6[best_j6])
best_g6 = pd.DataFrame(best_g6)
best_g6 = best_g6.round(4)

best_g7 = np.array(g7[best_j7])
best_g7 = pd.DataFrame(best_g7)
best_g7 = best_g7.round(4)

best_g8 = np.array(g8[best_j8])
best_g8 = pd.DataFrame(best_g8)
best_g8 = best_g8.round(4)

best_g9 = np.array(g9[best_j9])
best_g9 = pd.DataFrame(best_g9)
best_g9 = best_g9.round(4)

In [75]:
# Save cluster representatives
with pd.ExcelWriter("prototypes-zer.xlsx") as writer:
    best_g3.to_excel(writer, sheet_name="m=1.3")
    best_g4.to_excel(writer, sheet_name="m=1.4")
    best_g5.to_excel(writer, sheet_name="m=1.5")
    best_g6.to_excel(writer, sheet_name="m=1.6")
    best_g7.to_excel(writer, sheet_name="m=1.7")
    best_g8.to_excel(writer, sheet_name="m=1.8")
    best_g9.to_excel(writer, sheet_name="m=1.9")

In [76]:
# Width parameters
best_s3 = np.array(s3[best_j3])
best_s3 = pd.DataFrame(best_s3)
best_s3 = best_s3.round(4)

best_s4 = np.array(s4[best_j4])
best_s4 = pd.DataFrame(best_s4)
best_s4 = best_s4.round(4)

best_s5 = np.array(s5[best_j5])
best_s5 = pd.DataFrame(best_s5)
best_s5 = best_s5.round(4)

best_s6 = np.array(s6[best_j6])
best_s6 = pd.DataFrame(best_s6)
best_s6 = best_s6.round(4)

best_s7 = np.array(s7[best_j7])
best_s7 = pd.DataFrame(best_s7)
best_s7 = best_s7.round(4)

best_s8 = np.array(s8[best_j8])
best_s8 = pd.DataFrame(best_s8)
best_s8 = best_s8.round(4)

best_s9 = np.array(s9[best_j9])
best_s9 = pd.DataFrame(best_s9)
best_s9 = best_s9.round(4)

In [77]:
# Save width parameters
with pd.ExcelWriter("width-zer.xlsx") as writer:
    best_s3.to_excel(writer, sheet_name="m=1.3")
    best_s4.to_excel(writer, sheet_name="m=1.4")
    best_s5.to_excel(writer, sheet_name="m=1.5")
    best_s6.to_excel(writer, sheet_name="m=1.6")
    best_s7.to_excel(writer, sheet_name="m=1.7")
    best_s8.to_excel(writer, sheet_name="m=1.8")
    best_s9.to_excel(writer, sheet_name="m=1.9")